In [1]:
import pandas as pd

unmatch_data = pd.read_csv('unmatch_shops.csv')

In [2]:
# data = data.groupby('shop_index').sum()
unmatch_data.describe()
unmatch_data = unmatch_data.drop(columns=['Unnamed: 0','decorated_indicator'])
unmatch_data

X = unmatch_data.drop("masked_order",1)
y = unmatch_data["masked_order"]


In [3]:
#get unmatched shop features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

#apply SelectKBest class to extract top 5 best features
bestfeatures = SelectKBest(score_func=f_regression, k=5)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(5,'Score'))  #print 5 best features

                            Specs          Score
12         masked_item_impression  192094.743032
16         masked_other_tab_click   17161.243434
9                shop_L180D_order    7017.785715
8            shop_follower_number    4716.130794
14  masked_shop_click_from_search     141.479360


C:\Users\65843\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:301: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\65843\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:306: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom


In [4]:
#get features of whitelist shops
w_df = pd.read_csv('whitelist_filled_latest.csv')
w_df = w_df.drop(['Unnamed: 0','decorated_indicator'],1)
# w_df = w_df.groupby('shop_index').mean()
w_df

w_df = w_df.drop(['masked_campaign_tab_click'],1)
w_x = w_df.drop("masked_order",1)
w_y = w_df["masked_order"]

#apply SelectKBest class to extract top 5 best features
bestfeatures = SelectKBest(score_func=f_regression, k=5)
fit = bestfeatures.fit(w_x,w_y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(w_x.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(5,'Score'))  #print 5 best features


                            Specs        Score
14  masked_shop_click_from_search  2683.201382
15         masked_other_tab_click  1834.249525
12         masked_item_impression   589.313547
8            shop_follower_number   500.935029
9                shop_L180D_order   413.186404


In [5]:
#calculate the coefficient 
feature = featureScores.nlargest(5,'Score')
feature['Score'] = feature['Score']/(feature['Score'].sum())
feature

,Specs,Score
14,masked_shop_click_from_search,0.445649
15,masked_other_tab_click,0.304648
12,masked_item_impression,0.097878
8,shop_follower_number,0.083200
9,shop_L180D_order,0.068626


In [6]:
#calculate the score of whitelist shops
features = list(feature['Specs'])
scores = list(feature['Score'])
equation = ''
for i in range(len(features)):
    equation  += str(scores[i]) +'*'+str(features[i])+'+'
equation = 'Score='+ equation[:len(equation)-1]
equation
wf_process = w_df[features]
wf_process = wf_process.eval(equation)
pd.set_option('display.float_format',lambda x : '%.2f' % x)
# wf_process.to_csv('whitelist_with_score.csv')
wf_process 




,masked_shop_click_from_search,masked_other_tab_click,masked_item_impression,shop_follower_number,shop_L180D_order,Score
0,225.00,10299.00,23710764289.00,82005.00,156119.00,2320787993.63
1,361.00,9133.00,25607040484.00,82005.00,156119.00,2506391822.88
2,289.00,12598.00,21261139344.00,82005.00,156119.00,2081023812.28
3,484.00,8671.00,17363069361.00,82005.00,156119.00,1699486581.74
4,400.00,9254.00,20385271729.00,82005.00,156119.00,1995294486.70
...,...,...,...,...,...,...
7499,25.00,1631.00,19390562500.00,183561.00,197719.00,1897942940.46
7500,16.00,1400.00,16337696761.00,183561.00,197719.00,1599133825.36
7501,49.00,1686.00,22693012164.00,183561.00,197719.00,2221180837.01
7502,9.00,1400.00,18531104641.00,183561.00,197719.00,1813820664.23


In [19]:
import numpy as np
wf_process['Score'].describe()
threshold = np.percentile(wf_process['Score'],0.4)
threshold  

120.41541514041245

In [20]:
#get score for all unmatched shops
# unmatch_data.set_index(["shop_index"], inplace=True)
unmatch_data
un_data_process = unmatch_data[features]
un_data_process = un_data_process.eval(equation)
un_data_process
# data_process.to_csv('unmatch_with_score.csv')
un_data_process['Score'].describe()

count         111720.00
mean        41370823.33
std       2030310290.62
min              114.22
25%              524.16
50%            20026.26
75%           214157.10
max     496252535290.79
Name: Score, dtype: float64

In [27]:
select_shop = un_data_process[un_data_process['Score']>threshold]

select_shop = select_shop.groupby('shop_index').mean()
select_shop

,masked_shop_click_from_search,masked_other_tab_click,masked_item_impression,shop_follower_number,shop_L180D_order,Score
shop_index,,,,,,
119700,291.03,113.00,178146336.57,58590,25079,17443404.78
215775,54.21,262.29,183822156.93,15684,13197,17994498.71
406305,17.06,208.86,59091233.79,2139,1757,5784114.09
505260,5.59,27.69,25142.29,2205,1848,2782.08
1004355,12.74,39.76,1145190.86,1437,6138,112647.81
...,...,...,...,...,...,...
1484237745,2.98,20.91,3502.00,972,405,459.13
1484964585,9.06,31.08,393613.33,972,405,38648.34
1486048005,1.76,17.97,1109.07,972,405,223.48


In [24]:
#score for unmatched data
un_data_process['Score'].describe()

count         111720.00
mean        41370823.33
std       2030310290.62
min              114.22
25%              524.16
50%            20026.26
75%           214157.10
max     496252535290.79
Name: Score, dtype: float64

In [25]:
#perform equation on matched shops
match_df = pd.read_csv('match_with_label.csv')
match_df = match_df.drop(['Unnamed: 0','decorated_indicator'],1)
match_df = match_df.drop(['masked_campaign_tab_click'],1)
match_df



match_process = match_df[features]
match_process = match_process.eval(equation)
pd.set_option('display.float_format',lambda x : '%.2f' % x)
# wf_process.to_csv('whitelist_with_score.csv')
match_process 
#记得加进去datecolumn


,masked_shop_click_from_search,masked_other_tab_click,masked_item_impression,shop_follower_number,shop_L180D_order,Score
0,1.42,21.53,25.00,975,405,118.55
1,1.00,14.00,36.00,975,405,117.15
2,1.47,21.53,121.00,975,405,127.97
3,1.00,25.58,9.00,975,405,118.03
4,1.00,19.79,225.00,975,405,137.41
...,...,...,...,...,...,...
252527,1.00,23.26,0.00,999,405,118.44
252528,1.00,23.26,1.00,999,405,118.54
252529,1.00,23.26,4.00,999,405,118.83
252530,1.00,23.26,0.00,999,405,118.44


In [26]:
#score of matched shops
match_process['Score'].describe()

count           252532.00
mean         496375567.69
std        88229769931.21
min                113.37
25%                121.09
50%                198.02
75%               4818.65
max     33719730177769.17
Name: Score, dtype: float64

In [27]:
#score of whitelist shop
wf_process['Score'].describe()

count             7504.00
mean        9291614023.74
std       172574113786.31
min                119.28
25%             515177.86
50%           13457752.54
75%          170504258.71
max     10606572483257.32
Name: Score, dtype: float64

In [28]:
#score for unmatched data
un_data_process['Score'].describe()

count         111720.00
mean        41370823.33
std       2030310290.62
min              114.22
25%              524.16
50%            20026.26
75%           214157.10
max     496252535290.79
Name: Score, dtype: float64

In [29]:
select_shop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7532 entries, 119700 to 1486803240
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   masked_shop_click_from_search  7532 non-null   float64
 1   masked_other_tab_click         7532 non-null   float64
 2   masked_item_impression         7532 non-null   float64
 3   shop_follower_number           7532 non-null   int64  
 4   shop_L180D_order               7532 non-null   int64  
 5   Score                          7532 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 411.9 KB
